In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
## Load in state names, abbreviations and FIPS codes
stateabbr=pd.read_csv('msc_data_raw/stateabbrevations.csv')
StateFIPS=pd.read_excel('msc_data_raw/state-geocodes-v2016.xls')
stateabbr=stateabbr.set_index('State')
StateFIPS.columns=StateFIPS.iloc[3]
StateFIPS=StateFIPS.iloc[4:,2:]
StateFIPS.columns= ['State FIPS', 'State']
StateFIPS=StateFIPS.set_index('State')
Stateinfo=pd.concat([stateabbr, StateFIPS],axis=1, sort=False)
#Stateinfo

In [3]:
## Getting the election results and selection the useful ones
votes=pd.read_csv('msc_data_raw/election results 2000-2020.csv' )

#isolating the dems
votesdem=votes[votes['party']== 'DEMOCRAT']#.dropna()
votesdem=votesdem.loc[votesdem['totalvotes']>0.5]
votesdem=votesdem.loc[votesdem['county_fips']>2]
votesdem=votesdem.fillna(0)
#isolating the reps
votesrep=votes[votes['party']== 'REPUBLICAN']#.dropna()
votesrep=votesrep.loc[votesrep['totalvotes']>0.5]
votesrep=votesrep.loc[votesrep['county_fips']>2]
votesrep=votesrep.fillna(0)

#fixing the FIPS and setting them as well as the years as index

fips=votesdem['county_fips'].copy()
fipslist=[]

for i in fips:
    if len(str(round(i)))==4:
        fipslist.append('0'+str(round(i)))
    else:
        fipslist.append(str(round(i)))
votesdem['FIPS']=fipslist
votesdem=votesdem.set_index(['year','FIPS'])

fips=votesrep['county_fips'].copy()
fipslist=[]
for i in fips:
    if len(str(round(i)))==4:
        fipslist.append('0'+str(round(i)))
    else:
        fipslist.append(str(round(i)))
votesrep['FIPS']=fipslist
votesrep=votesrep.set_index(['year','FIPS'])
#delete duplicates due to several voting methodes especially in 2020

for i in votesdem.index.unique():
    if len(votesdem.loc[i])>1:
        votesdem.loc[i]['candidatevotes'] = sum(votesdem.loc[i]['candidatevotes'])
selection=np.array(votesdem['mode']=='TOTAL')
votesdem=votesdem.loc[selection]

for i in votesrep.index.unique():
    if len(votesrep.loc[i])>1:
        votesrep.loc[i]['candidatevotes'] = sum(votesrep.loc[i]['candidatevotes'])
selection= np.array(votesrep['mode']=='TOTAL')
votesrep=votesrep.loc[selection]

# use only the usefull columns

votesdem=votesdem.loc[:,['candidatevotes','totalvotes']]
votesrep=votesrep.loc[:,['candidatevotes','totalvotes']]
votesdem.columns.names = ['variables']
votesrep.columns.names =[ 'variables']
#calculating the percentages

votesdem['percentage of total']=votesdem['candidatevotes']/votesdem['totalvotes']
votesrep['percentage of total']=votesrep['candidatevotes']/votesrep['totalvotes']

votesdem['share of dem and rep']=np.array(votesdem['percentage of total'])/(np.array(votesdem['percentage of total'])+np.array(votesrep['percentage of total']))
votesrep['share of dem and rep']=np.array(votesrep['percentage of total'])/(np.array(votesdem['percentage of total'])+np.array(votesrep['percentage of total']))

votesdem['third party']=1-np.array(votesdem['percentage of total'])-np.array(votesrep['percentage of total'])
votesrep['third party']=1-np.array(votesdem['percentage of total'])-np.array(votesrep['percentage of total'])

votesdem=votesdem.unstack(level='year')
votesrep=votesrep.unstack(level='year')

#fix some missing values from 2020
results2020=pd.read_csv('msc_data_raw/2020_US_County_Level_Presidential_Results.csv')
fipslist=[]

for i in results2020['county_fips']:
    if len(str(round(i)))==4:
        fipslist.append('0'+str(round(i)))
    else:
        fipslist.append(str(round(i)))
results2020['county_fips']=fipslist
results2020=results2020.set_index('county_fips')
results2020
for i in votesdem[votesdem.isna().any(axis=1)].index[1:]:
    if i not in ['36000','51515']:
        votesdem.loc[i].iloc[5]=results2020.loc[i,'votes_dem']
        votesrep.loc[i].iloc[5]=results2020.loc[i,'votes_gop']
        votesdem.loc[i].iloc[11]=results2020.loc[i,'total_votes']
        votesrep.loc[i].iloc[11]=results2020.loc[i,'total_votes']
        votesdem.loc[i].iloc[17]=results2020.loc[i,'per_dem']
        votesrep.loc[i].iloc[17]=results2020.loc[i,'per_gop']
        votesdem.loc[i].iloc[23]=results2020.loc[i,'per_dem']
        votesrep.loc[i].iloc[23]=results2020.loc[i,'per_gop']
        votesdem.loc[i].iloc[-1]=0
        votesrep.loc[i].iloc[-1]=0

<ipython-input-3-54f5a937a0bc>:40: PerformanceWarning: indexing past lexsort depth may impact performance.
  if len(votesdem.loc[i])>1:
<ipython-input-3-54f5a937a0bc>:41: PerformanceWarning: indexing past lexsort depth may impact performance.
  votesdem.loc[i]['candidatevotes'] = sum(votesdem.loc[i]['candidatevotes'])
<ipython-input-3-54f5a937a0bc>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  votesdem.loc[i]['candidatevotes'] = sum(votesdem.loc[i]['candidatevotes'])
<ipython-input-3-54f5a937a0bc>:46: PerformanceWarning: indexing past lexsort depth may impact performance.
  if len(votesrep.loc[i])>1:
<ipython-input-3-54f5a937a0bc>:47: PerformanceWarning: indexing past lexsort depth may impact performance.
  votesrep.loc[i]['candidatevotes'] =

In [4]:
##cleaning SCI data for missing counties
SCI=pd.read_csv('msc_data_raw/county_county_aug2020.tsv', sep='\t').dropna()
fipslist=[]
notused1=[]
fips=votesdem.index
for i in SCI.loc[:,'user_loc']:
    if len(str(round(i)))==4:
        fipslist.append('0'+str(round(i)))
    else:
        fipslist.append(str(round(i)))
    if fipslist[-1] in fips:
        notused1.append(False)
    else:
        notused1.append(True)
SCI.loc[:,'user_loc']=fipslist
fipslist=[]
notused2=[]
for i in SCI.loc[:,'fr_loc']:
    temp=str(round(i))
    if len(temp)==4:
        fipslist.append('0'+temp)
    else:
        fipslist.append(temp)
    if fipslist[-1] in fips:
        notused2.append(False)
    else:
        notused2.append(True)
SCI.loc[:,'fr_loc']=fipslist
notused=np.array(notused1)+np.array(notused2)
SCI=SCI.drop(SCI.index[notused])
SCI=SCI.set_index(['user_loc','fr_loc'])
SCI=SCI.unstack('fr_loc')
SCI=SCI['scaled_sci']
SCI.to_csv('msc_thesis/data/SCI')

In [5]:
## calculation the exposure
SCIvotescorrecteddem=pd.DataFrame(index=SCI.index)
SCIvotesdem=pd.DataFrame(index=SCI.index)
SCIvotersdem=pd.DataFrame(index=SCI.index)

#generation an empty dataframe
for j in [2000,2004,2008,2012,2016,2020]:
    SCIvotesdem[j]=np.repeat(np.nan,len(SCI.index))
    SCIvotescorrecteddem[j]=np.repeat(np.nan,len(SCI.index))
SCIvotesrep=SCIvotesdem.copy()
SCIvotersrep= SCIvotescorrecteddem.copy()   
for i in SCI.index:
    for j in [2000,2004,2008,2012,2016,2020]:
        SCIvotesdem.loc[i,j]=sum((SCI.loc[i]*votesdem.loc[:,'candidatevotes'].loc[:,j]).dropna())
        SCIvotersdem.loc[i,j]=sum((SCI.loc[i]*votesdem.loc[:,'totalvotes'].loc[:,j]).dropna())
        SCIvotesrep.loc[i,j]=sum((SCI.loc[i]*votesrep.loc[:,'candidatevotes'].loc[:,j]).dropna())
        SCIvotersrep.loc[i,j]=sum((SCI.loc[i]*votesrep.loc[:,'totalvotes'].loc[:,j]).dropna())
exposuredem=SCIvotesdem/SCIvotersdem 
SCIvotersdem.to_csv('msc_thesis/data/eligible friends')
exposurerep=SCIvotesrep/SCIvotersrep     
 

In [ ]:
## adding the voteshares and exposure to the cleaned dataset
total_time_data=pd.concat([votesdem['share of dem and rep'],votesrep['share of dem and rep'],votesrep['third party'],exposurerep,exposuredem],axis=1)
total_time_data.columns=[np.repeat(['dem_votesshare','rep_voteshare','third party vote','exposure_rep','exposure_dem'],6),
                        [2000,2004,2008,2012,2016,2020,2000,2004,2008,2012,2016,2020,2000,2004,2008,2012,2016,2020,
                        2000,2004,2008,2012,2016,2020,2000,2004,2008,2012,2016,2020]]
total_time_data=total_time_data.dropna()

In [ ]:
#unemployment data
df_unemployment=pd.read_excel('msc_data_raw/Unemployment2000-2020.xlsx' )
df_unemployment.columns=df_unemployment.iloc[3]
df_unemployment=df_unemployment.dropna()
df_unemployment=df_unemployment.iloc[1:,:]
df_unemployment=df_unemployment.set_index('FIPS_Code')
df_unemployment.index.names=['FIPS']

unemployment_rate_list=[]
for i in df_unemployment.columns:
    if 'rate' in i:
        if i[-4:] in ['2000','2004','2008','2012','2016','2019']:
            unemployment_rate_list.append(i)
df_unemployment=df_unemployment.loc[:,unemployment_rate_list]
df_unemployment.columns=[['unemployment rate','unemployment rate','unemployment rate','unemployment rate','unemployment rate','unemployment rate'],
                        [2000,2004,2008,2012,2016,2020]]
df_unemployment


In [ ]:
## getting the age and racial data, this is split in the two seperate decades
## first the age groups 
df_raceandage10till20=pd.read_csv('msc_data_raw/racial and age composition 2010-2019.csv', encoding = 'utf-16')
#correcting the FIPS codes
fipslist=[]
fipslistcounty=[]
for i in range(len(df_raceandage10till20)):
    if len(str(df_raceandage10till20['COUNTY'].iloc[i]))==1:
        fipslistcounty.append('00' +str(df_raceandage10till20['COUNTY'].iloc[i]))
    elif len(str(df_raceandage10till20['COUNTY'].iloc[i]))==2:
        fipslistcounty.append('0' +str(df_raceandage10till20['COUNTY'].iloc[i]))
    elif len(str(df_raceandage10till20['COUNTY'].iloc[i]))==3:
        fipslistcounty.append(str(df_raceandage10till20['COUNTY'].iloc[i]))
for i in range(len(df_raceandage10till20)):
    if len(str(df_raceandage10till20['STATE'].iloc[i]))==1:
        fipslist.append('0' +str(df_raceandage10till20['STATE'].iloc[i])+fipslistcounty[i])
    elif len(str(df_raceandage10till20['STATE'].iloc[i]))==2:
        fipslist.append(str(df_raceandage10till20['STATE'].iloc[i])+fipslistcounty[i])
df_raceandage10till20['FIPS']=fipslist
df_raceandage10till20['County_FIPS']=fipslistcounty
df_ages1020=df_raceandage10till20

#selecting the election years 
election_years=np.array(df_ages1020['YEAR']==5)+np.array(df_ages1020['YEAR']==9)+np.array(df_ages1020['YEAR']==12)

df_ages1020=df_ages1020.loc[election_years]
years=[]
for i in range(len(df_ages1020['YEAR'])//57):
    years.extend(np.repeat(2012,19))
    years.extend(np.repeat(2016,19))
    years.extend(np.repeat(2020,19))
df_ages1020['YEARS']=years
df_ages1020=df_ages1020.loc[:,['FIPS','YEARS','AGEGRP','TOT_POP']]

eligible=[]
yeartemp=[]
FIPStemp=[]
for i in df_ages1020['FIPS'].unique():
    temp=df_ages1020.loc[df_ages1020['FIPS']==i]
    for j in [2012,2016,2020]:
        temp2=temp.loc[df_ages1020['YEARS']==j]
               
        FIPStemp.append(i)
        eligible.append(temp2.iloc[0,3]-temp2.iloc[1,3])
    yeartemp.extend([2012,2016,2020])   
eligiblevoter10till20=pd.DataFrame(eligible,index=[yeartemp,FIPStemp])
eligiblevoter10till20.index.names=['YEARS','FIPS']
eligiblevoter10till20=eligiblevoter10till20.unstack('YEARS')
eligiblevoter10till20.columns=pd.MultiIndex.from_tuples([(  'eligible voters', 2012),
            (  'eligible voters', 2016),
            (  'eligible voters', 2020),
            ],
           names=[ 'agegroups', 'YEARS'])
#sorting the data in the correct bin
yeartemp=[]
FIPStemp=[]
agegrouptemp=[]
percentagetemp=[]
for i in df_ages1020['FIPS'].unique():
    temp=df_ages1020.loc[df_ages1020['FIPS']==i]
    for j in [2012,2016,2020]:
        temp2=temp.loc[df_ages1020['YEARS']==j]
        temp2=temp2.loc[:,'TOT_POP']/temp2.iloc[0,3]
        yeartemp.extend(np.repeat(j,4))
        FIPStemp.extend(np.repeat(i,4))
        agegrouptemp.extend(['0-19 years old','20-29 years old','30-44 years old','45-64 years old'])
        percentagetemp.append(sum(temp2.iloc[1:5]))
        percentagetemp.append(sum(temp2.iloc[5:7]))
        percentagetemp.append(sum(temp2.iloc[7:10]))
        percentagetemp.append(sum(temp2.iloc[10:14]))
        
df_agegroups1020=pd.DataFrame(percentagetemp,index=[yeartemp,FIPStemp,agegrouptemp])
df_agegroups1020.index.names=['YEARS','FIPS','agegroups']
df_agegroups1020=df_agegroups1020.sort_index(axis=0,level=2)

df_agegroups1020=df_agegroups1020.unstack(['agegroups','YEARS'])
df_agegroups1020.columns=pd.MultiIndex.from_tuples([(  '0-19 years old', 2012),
            (  '0-19 years old', 2016),
            (  '0-19 years old', 2020),
            ( '20-29 years old', 2012),
            ( '20-29 years old', 2016),
            ( '20-29 years old', 2020),
            ( '30-44 years old', 2012),
            ( '30-44 years old', 2016),
            ( '30-44 years old', 2020),
            ( '45-64 years old', 2012),
            ( '45-64 years old', 2016),
            ('45-64 years old', 2020)],
           names=[ 'agegroups', 'YEARS'])
#now the same for the 2000-2010
df_ages00till10=pd.read_csv('msc_data_raw/age composition 2000-2010.csv').dropna()
fipslist=[]
fipslistcounty=[]
for i in range(len(df_ages00till10)):
    if len(str(df_ages00till10['COUNTY'].iloc[i]))==1:
        fipslistcounty.append('00' +str(df_ages00till10['COUNTY'].iloc[i]))
    elif len(str(df_ages00till10['COUNTY'].iloc[i]))==2:
        fipslistcounty.append('0' +str(df_ages00till10['COUNTY'].iloc[i]))
    elif len(str(df_ages00till10['COUNTY'].iloc[i]))==3:
        fipslistcounty.append(str(df_ages00till10['COUNTY'].iloc[i]))
for i in range(len(df_ages00till10)):
    if len(str(df_ages00till10['STATE'].iloc[i]))==1:
        fipslist.append('0' +str(df_ages00till10['STATE'].iloc[i])+fipslistcounty[i])
    elif len(str(df_ages00till10['STATE'].iloc[i]))==2:
        fipslist.append(str(df_ages00till10['STATE'].iloc[i])+fipslistcounty[i])
df_ages00till10['FIPS']=fipslist
df_ages00till10=df_ages00till10.loc[df_ages00till10.loc[:,'SEX']==0]
df_ages00till10=df_ages00till10[['AGEGRP','POPESTIMATE2000','POPESTIMATE2004','POPESTIMATE2008','FIPS']]
df_ages00till10.columns=['AGEGRP',2000,2004,2008,'FIPS']

temp1 =df_ages00till10.loc[df_ages00till10.loc[:,'AGEGRP']==0].set_index('FIPS').drop('AGEGRP',axis=1)
temp2 =df_ages00till10.loc[df_ages00till10.loc[:,'AGEGRP']==1].set_index('FIPS').drop('AGEGRP',axis=1)
eligiblevoter00till10=pd.DataFrame(index=temp1.index,columns=[np.repeat(['totalpop','underage','eligible voters'],3),
                                                              [2000,2004,2008,2000,2004,2008,2000,2004,2008]])
eligiblevoter00till10['totalpop']=temp1
eligiblevoter00till10['underage']=temp2
eligiblevoter00till10['eligible voters']=eligiblevoter00till10['totalpop']-eligiblevoter00till10['underage']

df_eligibevoters=pd.concat([eligiblevoter00till10['eligible voters'],eligiblevoter10till20],axis=1).dropna()
df_eligibevoters.columns=[np.repeat('eligible voters',6),[2000,2004,2008,2012,2016,2020]]
voterturnout=pd.concat([votesdem['totalvotes'],df_eligibevoters],axis=1).dropna()
voterturnout.columns=[np.repeat(['total votes','eligible voters'],6),
[2000,2004,2008,2012,2016,2020,2000,2004,2008,2012,2016,2020]]
for i in [2000,2004,2008,2012,2016,2020]:
    voterturnout['voter turnout '+str(i)]=voterturnout['total votes',i]/voterturnout['eligible voters',i]
voterturnout.columns=[np.repeat(['total votes','eligible voters','voter turnout'],6),
[2000,2004,2008,2012,2016,2020,2000,2004,2008,2012,2016,2020,2000,2004,2008,2012,2016,2020]]

#save eligible voters for the simulation
voterturnout['eligible voters'].to_csv('msc_thesis/data/eligible voters')

df_ages00till10=df_ages00till10.set_index(['FIPS']).dropna()

yeartemp=[]
FIPStemp=[]
agegrouptemp=[]
percentagetemp=[]
for i in df_ages00till10.index.unique():
    temp=df_ages00till10.loc[i]
    temp2=temp.loc[:,[2000,2004,2008]]
    temp2=temp2/temp2.iloc[0]
    FIPStemp.extend(np.repeat(i,4))
    agegrouptemp.extend(['0-19 years old','20-29 years old','30-44 years old','45-64 years old'])
    percentagetemp.append(np.sum(temp2.iloc[1:5],axis=0))
    percentagetemp.append(np.sum(temp2.iloc[5:7],axis=0))
    percentagetemp.append(np.sum(temp2.iloc[7:10],axis=0))
    percentagetemp.append(np.sum(temp2.iloc[10:14],axis=0))
    
df_ages00till10=pd.DataFrame(percentagetemp,index=[FIPStemp,agegrouptemp])
df_ages00till10.index.names=['FIPS','age groups']
df_ages00till10=df_ages00till10.unstack('age groups') 
df_ages00till10=pd.DataFrame(percentagetemp,index=[FIPStemp,agegrouptemp])
df_ages00till10.index.names=['FIPS','age groups']
df_ages00till10.columns.names=['YEARS']
df_ages00till10=df_ages00till10.unstack('age groups')
df_ages00till10=df_ages00till10.stack('YEARS').unstack('YEARS')
df_ages=pd.concat([df_ages00till10,df_agegroups1020],axis=1).dropna()
df_ages=df_ages.sort_index(axis=1,level='age groups')


#racial composition
df_allagesrace1020=df_raceandage10till20.loc[df_raceandage10till20['AGEGRP']==0]
election_years=np.array(df_allagesrace1020['YEAR']==5)+np.array(df_allagesrace1020['YEAR']==9)+np.array(df_allagesrace1020['YEAR']==12)
df_allagesrace1020=df_allagesrace1020.loc[election_years ]
years=[]
for i in range(len(df_allagesrace1020['YEAR'])//3):
    years.append(2012)
    years.append(2016)
    years.append(2020)
df_allagesrace1020['YEARS']=years
df_allagesrace1020=df_allagesrace1020.set_index(['FIPS','YEARS'])
df_allagesrace1020=df_allagesrace1020.loc[:,['TOT_POP','NHWA_MALE','NHWA_FEMALE','NHBA_MALE','NHBA_FEMALE',
                                             'NHIA_MALE','NHIA_FEMALE','NHAA_MALE','NHAA_FEMALE','NHNA_MALE',
                                             'NHNA_FEMALE','NHTOM_MALE','NHTOM_FEMALE','H_MALE','H_FEMALE']]
df_race1020=(df_allagesrace1020.T / df_allagesrace1020.TOT_POP).T
df_race1020=df_race1020.unstack('YEARS')
df_race1020=df_race1020.loc[:,['NHWA_MALE','NHWA_FEMALE','NHBA_MALE','NHBA_FEMALE',
                                       'NHIA_MALE','NHIA_FEMALE','NHAA_MALE','NHAA_FEMALE','NHNA_MALE','NHNA_FEMALE',
                                       'NHTOM_MALE','NHTOM_FEMALE','H_MALE','H_FEMALE']]
#2000-2010
df_raceandsex00till10=df_raceandsex00till10.loc[:,['FIPS','SEX','ORIGIN','RACE','POPESTIMATE2000','POPESTIMATE2004','POPESTIMATE2008']]
df_raceandsex00till10=df_raceandsex00till10.set_index('FIPS')
df_racefractions00till10=pd.DataFrame(columns=['POPESTIMATE2000','POPESTIMATE2004','POPESTIMATE2008'])
for i in df_raceandsex00till10.index.unique():
    election=['POPESTIMATE2000','POPESTIMATE2004','POPESTIMATE2008']
    temp=df_raceandsex00till10.loc[i,election]/df_raceandsex00till10.loc[i,election].iloc[0]
    df_racefractions00till10=np.concatenate([df_racefractions00till10,temp])
df_racefractions00till10= pd.DataFrame(df_racefractions00till10,index=fipslist,columns=['percentage 2000','percentage 2004','percentage 2008'])   
df_racefractions00till10=pd.DataFrame(df_racefractions00till10,index=fipslist,columns=['percentage 2000','percentage 2004','percentage 2008'])
df_raceandsex00till10=pd.DataFrame(np.concatenate([df_raceandsex00till10,df_racefractions00till10],axis=1)
            ,columns=['SEX','ORIGIN','RACE','POPESTIMATE2000','POPESTIMATE2004','POPESTIMATE2008',
                      2000,2004,2008]
            )
df_raceandsex00till10['FIPS']=fipslist
df_raceandsex00till10=df_raceandsex00till10.loc[:,['FIPS','SEX','ORIGIN','RACE',2000,2004,2008]]
df_raceandsex00till10=df_raceandsex00till10.set_index(['FIPS','SEX','ORIGIN','RACE'])
years=np.repeat([2000,2004,2008],63)

df_raceandsex00till10=df_raceandsex00till10.unstack(['SEX','ORIGIN','RACE'])
listofused=[]
listused2=[]
listofused.extend(np.repeat(False,29))
listofused.extend(np.repeat(True,7))
listofused.extend(np.repeat(False,14))
listofused.extend(np.repeat(True,7))
listofused.extend(np.repeat(False,6))
listused2.extend(listofused)
listused2.extend(listofused)
listused2.extend(listofused)
df_raceandsex00till10=df_raceandsex00till10.loc[:,listused2]
df_raceandsex00till10=df_raceandsex00till10.stack(['SEX','ORIGIN','RACE'])

df_raceandsex00till10=df_raceandsex00till10.sort_index(level='RACE')
df_raceandsex00till10=df_raceandsex00till10.unstack(['SEX','ORIGIN','RACE'])

df_raceandsex00till10.columns=[['H_MALE','H_FEMALE','NHWA_MALE','NHWA_FEMALE','NHBA_MALE','NHBA_FEMALE','NHIA_MALE','NHIA_FEMALE',
     'NHAA_MALE','NHAA_FEMALE','NHNA_MALE','NHNA_FEMALE','NHTOM_MALE','NHTOM_FEMALE',
              'H_MALE','H_FEMALE','NHWA_MALE','NHWA_FEMALE','NHBA_MALE','NHBA_FEMALE','NHIA_MALE','NHIA_FEMALE',
     'NHAA_MALE','NHAA_FEMALE','NHNA_MALE','NHNA_FEMALE','NHTOM_MALE','NHTOM_FEMALE',
              'H_MALE','H_FEMALE','NHWA_MALE','NHWA_FEMALE','NHBA_MALE','NHBA_FEMALE','NHIA_MALE','NHIA_FEMALE',
     'NHAA_MALE','NHAA_FEMALE','NHNA_MALE','NHNA_FEMALE','NHTOM_MALE','NHTOM_FEMALE'],np.repeat([2000,2004,2008],14)]
#combine both decades
df_race=pd.concat([df_raceandsex00till10,df_race1020],axis=1)
df_race.columns.names=['year','race']
df_race=df_race.stack('year')
df_race=df_race[sorted(df_race.columns)]
df_race=df_race.stack('race')
df_race=df_race.unstack(['year','race'])


total_time_data=pd.concat([total_time_data,df_ages,voterturnout.iloc[:,12:],df_race],axis=1)
total_time_data=total_time_data.dropna()

In [ ]:
##Poverty and median income data are seperated datasets so there the same proces 
##is repeated 6 times to clean the whole data set

#2020
df_poverty2019=pd.read_excel('msc_data_raw/median income and poverty/est19all.xls' )
df_poverty2019.columns=df_poverty2019.iloc[2]
df_poverty2019=df_poverty2019.iloc[3:,:-6]
df_poverty2019=df_poverty2019.loc[df_poverty2019['County FIPS Code']!= '000']
df_poverty2019['FIPS']=df_poverty2019['State FIPS Code']+df_poverty2019['County FIPS Code']
df_poverty2019=df_poverty2019.set_index('FIPS')
df_poverty2019=df_poverty2019.loc[:,['Poverty Percent, All Ages','Poverty Percent, Age 0-17','Median Household Income']]

#2016
df_poverty2016=pd.read_excel('msc_data_raw/median income and poverty/est16all.xls' )
df_poverty2016.columns=df_poverty2016.iloc[2]
df_poverty2016=df_poverty2016.iloc[3:,:-6]
df_poverty2016=df_poverty2016.loc[df_poverty2016['County FIPS Code']!= '000']
df_poverty2016['FIPS']=df_poverty2016['State FIPS Code']+df_poverty2016['County FIPS Code']
df_poverty2016=df_poverty2016.set_index('FIPS')
df_poverty2016=df_poverty2016.loc[:,['Poverty Percent, All Ages','Poverty Percent, Age 0-17','Median Household Income']]

#2012
df_poverty2012=pd.read_excel('msc_data_raw/median income and poverty/est12all.xls' )
df_poverty2012.columns=df_poverty2012.iloc[1]
df_poverty2012=df_poverty2012.iloc[2:,:-6]
df_poverty2012=df_poverty2012.loc[df_poverty2012['County FIPS Code']!= '000']
df_poverty2012['FIPS']=df_poverty2012['State FIPS Code']+df_poverty2012['County FIPS Code']
df_poverty2012=df_poverty2012.set_index('FIPS')
df_poverty2012=df_poverty2012.loc[:,['Poverty Percent, All Ages','Poverty Percent, Under Age 18','Median Household Income']]

#2008
df_poverty2008=pd.read_excel('msc_data_raw/median income and poverty/est08all.xls' )
df_poverty2008.columns=df_poverty2008.iloc[1]
df_poverty2008=df_poverty2008.iloc[2:-3,:-6]
fipslist=[]
for i in range(len(df_poverty2008)):
    if str(df_poverty2008['County FIPS'].iloc[i])=='0':
        fipslist.append('000')
    elif len(str(df_poverty2008['County FIPS'].iloc[i]))==1:
        fipslist.append(df_poverty2008['State FIPS'].iloc[i]+'00' +str(df_poverty2008['County FIPS'].iloc[i]))
    elif len(str(df_poverty2008['County FIPS'].iloc[i]))==2:
        fipslist.append(df_poverty2008['State FIPS'].iloc[i]+'0' +str(df_poverty2008['County FIPS'].iloc[i]))
    elif len(str(df_poverty2008['County FIPS'].iloc[i]))==3:
        fipslist.append(df_poverty2008['State FIPS'].iloc[i]+str(df_poverty2008['County FIPS'].iloc[i]))
df_poverty2008['FIPS']=fipslist
df_poverty2008=df_poverty2008.loc[df_poverty2008['FIPS'] != '000']
df_poverty2008=df_poverty2008.set_index('FIPS')
df_poverty2008=df_poverty2008.loc[:,['Poverty Percent All Ages','Poverty Percent Under Age 18','Median Household Income']]

#2004
df_poverty2004=pd.read_excel('msc_data_raw/median income and poverty/est04all.xls' )
df_poverty2004.columns=df_poverty2004.iloc[0]
df_poverty2004=df_poverty2004.iloc[2:,:-6]
fipslistcounty=[]
fipslist=[]
for i in range(len(df_poverty2004)):
    if len(str(df_poverty2004['County FIPS'].iloc[i]))==1:
        fipslistcounty.append('00' +str(df_poverty2004['County FIPS'].iloc[i]))
    elif len(str(df_poverty2004['County FIPS'].iloc[i]))==2:
        fipslistcounty.append('0' +str(df_poverty2004['County FIPS'].iloc[i]))
    elif len(str(df_poverty2004['County FIPS'].iloc[i]))==3:
        fipslistcounty.append(str(df_poverty2004['County FIPS'].iloc[i]))
for i in range(len(df_poverty2004)):
    if len(str(df_poverty2004['State FIPS'].iloc[i]))==1:
        fipslist.append('0' +str(df_poverty2004['State FIPS'].iloc[i])+fipslistcounty[i])
    elif len(str(df_poverty2004['State FIPS'].iloc[i]))==2:
        fipslist.append(str(df_poverty2004['State FIPS'].iloc[i])+fipslistcounty[i])
df_poverty2004['FIPS']=fipslist
df_poverty2004['County_FIPS']=fipslistcounty
df_poverty2004=df_poverty2004.loc[df_poverty2004['County_FIPS']!= '000']

df_poverty2004=df_poverty2004.set_index('FIPS')
df_poverty2004=df_poverty2004.loc[:,['Poverty Percent All Ages','Poverty Percent Under Age 18','Median Household Income']]

#2000
df_poverty2000=pd.read_excel('msc_data_raw/2000-Census-by-county.xls' ).dropna()
df_poverty2000.columns=df_poverty2000.iloc[0]
df_childpoverty2000=pd.read_excel('msc_data_raw/childpoverty2000.xlsx' ).dropna()
df_medianincome2000=pd.read_excel('msc_data_raw/medianincome2000.xlsx' )
df_medianincome2000=df_medianincome2000.iloc[:,0:2].dropna()
df_medianincome2000.columns=['FIPS','median income']
df_medianincome2000=df_medianincome2000.loc[df_medianincome2000['FIPS'].astype(int)>99]

df_medianincome2000=df_medianincome2000.set_index('FIPS')
df_poverty2000=df_poverty2000.set_index('State/ County Codes')
df_poverty2000=df_poverty2000.iloc[1:,4]
df_poverty2000.index.names=['FIPS']
df_childpoverty2000=df_childpoverty2000.set_index(df_childpoverty2000.iloc[:,0])
df_childpoverty2000=df_childpoverty2000.iloc[:,1]
df_medianincome2000

df_poverty2000=pd.concat([df_childpoverty2000,df_poverty2000],axis=1)
fipslist=[]
for i in df_poverty2000.index:
    if len(str(i))==4:
        fipslist.append('0'+str(i))
    else:
        fipslist.append(str(i))
fipslist            

df_poverty2000=df_poverty2000.set_index(pd.Series(fipslist))
df_poverty2000=pd.concat([df_poverty2000,df_medianincome2000],axis=1).dropna()
df_poverty2000.columns=[[2000,2000,2000],['poverty','youth poverty','median income']]

#adding all 6 years together
df_poverty=pd.concat([df_poverty2000,df_poverty2004, df_poverty2008,df_poverty2012,df_poverty2016,df_poverty2019],axis=1)
df_poverty.columns=[['poverty','youth poverty','median income','poverty','youth poverty','median income',
                           'poverty','youth poverty','median income','poverty','youth poverty','median income',
                           'poverty','youth poverty','median income','poverty','youth poverty','median income'],
                         [2000,2000,2000,2004,2004,2004,2008,2008,2008,2012,2012,2012,2016,2016,2016,2020,2020,2020]]
df_poverty=pd.concat([df_poverty['poverty'],df_poverty['youth poverty'],df_poverty['median income']],axis=1)
df_poverty.columns=[['poverty','poverty','poverty','poverty','poverty','poverty',
                           'youth poverty','youth poverty','youth poverty','youth poverty','youth poverty','youth poverty',
                           'median income','median income','median income','median income','median income','median income'],
                         [2000,2004,2008,2012,2016,2020,2000,2004,2008,2012,2016,2020,2000,2004,2008,2012,2016,2020]]
df_poverty=df_poverty.dropna()
df_poverty.loc[:,['poverty','youth poverty']]=df_poverty.loc[:,['poverty','youth poverty']]/100
#adding the poverty data to the total data
data=data.drop(['poverty','youth poverty','median income'],axis=1)
total_time_data=pd.concat([data, df_poverty],axis=1)
total_time_data=total_time_data.dropna()

In [ ]:
# The homestates of the presidential candidate and their running mate and incumbancy
incumbantdem=[1,0,0,1,1,0]
incumbantrep=[0,1,1,0,0,1]
homestatedempres=['TN','MA','IL','IL','NY','DE']
homestatereppres=['TX','TX','AZ','MA','NY','FL']
homestatedemvp=['CT','NC','DE','DE','VA','CA']
homestaterepvp=['WY','WY','AK','WI','IN','IN']
df_legacydem=pd.DataFrame(list(zip(incumbantdem,homestatedempres,homestatedemvp)),
                          index=[2000,2004,2008,2012,2016,2020]
                          ,columns=['incumbant', 'homestate of president', 'homestate of vp'])
df_legacyrep=pd.DataFrame(list(zip(incumbantrep,homestatereppres,homestaterepvp)),
                          index=[2000,2004,2008,2012,2016,2020]
                          ,columns=['incumbant', 'homestate of president', 'homestate of vp'])
# restructuring this to the general data frame format
df_homestatepresdem=pd.DataFrame(index=total_time_data.index)
df_homestatevpdem=pd.DataFrame(index=total_time_data.index)

for i in df_legacydem.index:
    dempres=[]
    demvp=[]
    presstate=df_legacydem.loc[i,'homestate of president'] 
    vpstate=df_legacydem.loc[i,'homestate of vp']
    presFIPS=Stateinfo.loc[Stateinfo['Code']==presstate]['State FIPS']
    vpFIPS=Stateinfo.loc[Stateinfo['Code']==vpstate]['State FIPS']
    for j in total_time_data.index:
        if str(j)[0:2]==presFIPS.iloc[0]:
            dempres.append(1)
        else:
            dempres.append(0)
        if str(j)[0:2]==vpFIPS.iloc[0]:
            demvp.append(1)
        else:
            demvp.append(0)
                
    df_homestatepresdem[i]=np.array(dempres)
    df_homestatevpdem[i]=np.array(demvp)
df_homestatepresdem.columns=[['homestate of democratic president','homestate of democratic president','homestate of democratic president',
                          'homestate of democratic president','homestate of democratic president','homestate of democratic president',],
                          [2000,2004,2008,2012,2016,2020]]
df_homestatevpdem.columns=[['homestate of democratic vp','homestate of democratic vp','homestate of democratic vp',
                          'homestate of democratic vp','homestate of democratic vp','homestate of democratic vp',],
                          [2000,2004,2008,2012,2016,2020]]
df_homestatepresrep=pd.DataFrame(index=total_time_data.index)
df_homestatevprep=pd.DataFrame(index=total_time_data.index)

for i in df_legacydem.index:
    reppres=[]
    repvp=[]
    presstate=df_legacyrep.loc[i,'homestate of president'] 
    vpstate=df_legacyrep.loc[i,'homestate of vp']
    presFIPS=Stateinfo.loc[Stateinfo['Code']==presstate]['State FIPS']
    vpFIPS=Stateinfo.loc[Stateinfo['Code']==vpstate]['State FIPS']
    for j in total_time_data.index:
        if str(j)[0:2]==presFIPS.iloc[0]:
            reppres.append(1)
        else:
            reppres.append(0)
        if str(j)[0:2]==vpFIPS.iloc[0]:
            repvp.append(1)
        else:
            repvp.append(0)
                
    df_homestatepresrep[i]=np.array(reppres)
    df_homestatevprep[i]=np.array(repvp)
df_homestatepresrep.columns=[['homestate of republican president','homestate of republican president','homestate of republican president',
                          'homestate of republican president','homestate of republican president','homestate of republican president',],
                          [2000,2004,2008,2012,2016,2020]]
df_homestatevprep.columns=[['homestate of republican vp','homestate of republican vp','homestate of republican vp',
                          'homestate of republican vp','homestate of republican vp','homestate of republican vp',],
                          [2000,2004,2008,2012,2016,2020]]

df_incubantdem=pd.DataFrame(index=total_time_data.index)
for i in df_legacydem.index:
    df_incubantdem[i]=np.repeat(df_legacydem.loc[i,'incumbant'],len(total_time_data.index))
df_incubantdem.columns=[['democratic incumbant','democratic incumbant','democratic incumbant',
                         'democratic incumbant','democratic incumbant','democratic incumbant'],
                        [2000,2004,2008,2012,2016,2020]]
#Add to total data
total_time_data=pd.concat([total_time_data,df_incubantdem,df_homestatepresdem,df_homestatevpdem,df_homestatepresrep,df_homestatevprep],axis=1)
total_time_data=total_time_data.dropna()

In [ ]:
##Religious data consist out of the census of 2000 and 2010 this data is used to inter and extrapolate
#first for 2010
fipslist=[]
for i in range(len(df_religion10)):
    if len(str(df_religion10['FIPS'].iloc[i]))==4:
        fipslist.append('0' +str(df_religion10['FIPS'].iloc[i]))
    elif len(str(df_religion10['FIPS'].iloc[i]))==5:
        fipslist.append(str(df_religion10['FIPS'].iloc[i]))
df_religion10['FIPS']=fipslist
df_religion10=df_religion10.set_index('FIPS')
#religions that where selected 
religions10=['EVANRATE','CATHRATE','ORTHRATE','AMSHRATE','MSLMRATE','SALVRATE','SBCRATE','UMCRATE']
df_religionrate10=pd.DataFrame(index=df_religion10.index)
# the following religions consist out of more than one religious sub group
df_religionrate10['BAPTRATE']=np.array(df_religion10.loc[:,'ABARATE'])+np.array(df_religion10.loc[:,'ABCRATE'])
for i in religions10:
    df_religionrate10[i]=np.array(df_religion10.loc[:,i])
df_religionrate10['JEWRATE']=np.array(df_religion10.loc[:,'RJUDRATE'])+np.array(df_religion10.loc[:,'OJUDRATE'])+np.array(df_religion10.loc[:,'CJUDRATE'])

#for 2000
df_religion00=pd.read_csv('msc_data_raw/Religious Congregations and Membership Study, 2000 (Counties File).csv'
                          ,sep=';',decimal=',')
fipslist=[]
for i in range(len(df_religion00)):
    if len(str(df_religion00['FIP'].iloc[i]))==4:
        fipslist.append('0' +str(df_religion00['FIP'].iloc[i]))
    elif len(str(df_religion00['FIP'].iloc[i]))==5:
        fipslist.append(str(df_religion00['FIP'].iloc[i]))
df_religion00['FIPS']=fipslist
df_religion00=df_religion00.set_index('FIPS')
religions00=['EVANRT','CATHRT','ORTHRT','AMOTHRT','ISLAMRT','SALARRT','SBCRT','UMETHRT','JEWRT']
df_religionrate00=pd.DataFrame(index=df_religion00.index)
df_religionrate00['BAPTRT']=np.array(df_religion00.loc[:,'AMBAPRT'])+np.array(df_religion00.loc[:,'ABAPRT'])
for i in religions00:
    df_religionrate00[i]=np.array(df_religion00.loc[:,i])
    
#combine both years in one dataframe
df_religion=pd.concat([df_religionrate00,df_religionrate10],axis=1).dropna()

#using the inter and extrapolation to estimate the religions for the election years
df_religionrate=pd.DataFrame(index=df_religion00.index)
years=[]
for i in range(len(df_religionrate00.columns)):
    year00=np.array(df_religion.loc[:,df_religionrate00.columns[i]])
    year10=np.array(df_religion.loc[:,df_religionrate10.columns[i]])
    diff=year10-year00
    df_religionrate[df_religionrate10.columns[i]+'2000'] = year00
    df_religionrate[df_religionrate10.columns[i]+'2004'] = year00*6/10+year10*4/10
    df_religionrate[df_religionrate10.columns[i]+'2008'] = year00*2/10+year10*8/10
    temp=year10 +diff*2/10
    df_religionrate[df_religionrate10.columns[i]+'2012'] =  np.where(temp <0, 0, temp)
    temp=year10 +diff*6/10
    df_religionrate[df_religionrate10.columns[i]+'2016'] = np.where(temp <0, 0, temp)
    temp=year10 +diff
    df_religionrate[df_religionrate10.columns[i]+'2020'] = np.where(temp <0, 0, temp)
    years.append(2000)
    years.append(2004)
    years.append(2008)
    years.append(2012)
    years.append(2016)
    years.append(2020)
df_religionrate.columns=[np.repeat(df_religionrate10.columns,6),years]

#add to total data
total_time_data=pd.concat([total_time_data,df_religionrate],axis=1)
total_time_data=total_time_data.dropna()

In [ ]:
# education here the data that is availible is for 2000 and 2015-2019 so again inter and extrapolation is needed
df_education=pd.read_excel('msc_data_raw/Education.xls')
df_education.columns=df_education.iloc[3]
df_education=df_education.iloc[6:].dropna()

lesshighschool2015=np.array(df_education['Percent of adults with less than a high school diploma, 2015-19'])
highschool2015=np.array(df_education['Percent of adults with a high school diploma only, 2015-19'])
bsc2015=np.array(df_education["Percent of adults with a bachelor's degree or higher, 2015-19"])
lesshighschool2000=np.array(df_education['Percent of adults with less than a high school diploma, 2000'])
highschool2000=np.array(df_education['Percent of adults with a high school diploma only, 2000'])
bsc2000=np.array(df_education["Percent of adults with a bachelor's degree or higher, 2000"])
df_lesshighschool=pd.DataFrame(index=df_education['FIPS Code'])
df_lesshighschool[2000]=lesshighschool2000
df_lesshighschool[2004]=lesshighschool2000*11/15+lesshighschool2015*4/15
df_lesshighschool[2008]=lesshighschool2000*7/15+lesshighschool2015*8/15
df_lesshighschool[2012]=lesshighschool2000*3/15+lesshighschool2015*12/15
df_lesshighschool[2016]=lesshighschool2000*1/15+lesshighschool2015*14/15
df_lesshighschool[2020]=lesshighschool2015
df_lesshighschool.columns=[np.repeat('Less than highschool',6),df_lesshighschool.columns]
df_highschool=pd.DataFrame(index=df_education['FIPS Code'])
df_highschool[2000]=highschool2000
df_highschool[2004]=highschool2000*11/15+highschool2015*4/15
df_highschool[2008]=highschool2000*7/15+highschool2015*8/15
df_highschool[2012]=highschool2000*3/15+highschool2015*12/15
df_highschool[2016]=highschool2000*1/15+highschool2015*14/15
df_highschool[2020]=highschool2015
df_highschool.columns=[np.repeat('Highschool',6),df_highschool.columns]
df_bsc=pd.DataFrame(index=df_education['FIPS Code'])
df_bsc[2000]=bsc2000
df_bsc[2004]=bsc2000*11/15+bsc2015*4/15
df_bsc[2008]=bsc2000*7/15+bsc2015*8/15
df_bsc[2012]=bsc2000*3/15+bsc2015*12/15
df_bsc[2016]=bsc2000*1/15+bsc2015*14/15
df_bsc[2020]=bsc2015
df_bsc.columns=[np.repeat('Bachelor',6),df_bsc.columns]

#add to total data
total_time_data=pd.concat([total_time_data,df_lesshighschool,df_highschool,df_bsc],axis=1)
total_time_data=total_time_data.dropna()

In [ ]:
# using the rural-urban continuum code the sort counties in rural, metro and metro adjacent

df_rural13=pd.read_excel('msc_data_raw/ruralurbancodes2013.xls')
fipslist=[]
for i in range(len(df_rural13)):
    if len(str(df_rural13['FIPS'].iloc[i]))==4:
        fipslist.append('0' +str(df_rural13['FIPS'].iloc[i]))
    elif len(str(df_rural13['FIPS'].iloc[i]))==5:
        fipslist.append(str(df_rural13['FIPS'].iloc[i]))
df_rural13['FIPS']=fipslist
df_rural13=df_rural13.set_index('FIPS')
df_rural1020=pd.DataFrame(index=df_rural.index)
df_rural03=pd.read_excel('msc_data_raw/ruralurbancodes2003.xls')
fipslist=[]
for i in range(len(df_rural03)):
    if len(str(df_rural03['FIPS Code'].iloc[i]))==4:
        fipslist.append('0' +str(df_rural03['FIPS Code'].iloc[i]))
    elif len(str(df_rural03['FIPS Code'].iloc[i]))==5:
        fipslist.append(str(df_rural03['FIPS Code'].iloc[i]))
df_rural03['FIPS']=fipslist
df_rural03=df_rural03.set_index('FIPS')

df_rural0010=pd.DataFrame(index=df_rural.index)
df_rural0010[2000]=np.array(df_rural['2003 Rural-urban Continuum Code'])*1.3-np.array(df_rural['RUCC_2013'])*0.3<=3
df_rural0010[2004]=np.array(df_rural['2003 Rural-urban Continuum Code'])*0.9+np.array(df_rural['RUCC_2013'])*0.1<=3
df_rural0010[2008]=np.array(df_rural['2003 Rural-urban Continuum Code'])*0.5+np.array(df_rural['RUCC_2013'])*0.5<=3
df_rural1020[2012]=np.array(df_rural['2003 Rural-urban Continuum Code'])*0.1+np.array(df_rural['RUCC_2013'])*0.9<=3
df_rural1020[2016]=np.array(-df_rural['2003 Rural-urban Continuum Code'])*0.3+np.array(df_rural['RUCC_2013'])*1.3<=3
df_rural1020[2020]=np.array(-df_rural['2003 Rural-urban Continuum Code'])*0.7+np.array(df_rural['RUCC_2013'])*1.7<=3



df_rural0010[20002]=np.array(df_rural['2003 Rural-urban Continuum Code'])*1.3-np.array(df_rural['RUCC_2013'])*0.3<=6  -np.array(df_rural0010[2000])
df_rural0010[20042]=np.array(df_rural['2003 Rural-urban Continuum Code'])*0.9+np.array(df_rural['RUCC_2013'])*0.1<=6 -np.array(df_rural0010[2004])
df_rural0010[20082]=np.array(df_rural['2003 Rural-urban Continuum Code'])*0.5+np.array(df_rural['RUCC_2013'])*0.5<=6 -np.array(df_rural0010[2008])
df_rural1020[20122]=np.array(df_rural['2003 Rural-urban Continuum Code'])*0.1+np.array(df_rural['RUCC_2013'])*0.9<=6 -np.array(df_rural1020[2012])
df_rural1020[20162]=np.array(-df_rural['2003 Rural-urban Continuum Code'])*0.3+np.array(df_rural['RUCC_2013'])*1.3<=6-np.array(df_rural1020[2016])
df_rural1020[20202]=np.array(-df_rural['2003 Rural-urban Continuum Code'])*0.7+np.array(df_rural['RUCC_2013'])*1.7<=6-np.array(df_rural1020[2020])

df_rural1020.columns=[['metro','metro','metro','metro adjacent','metro adjacent','metro adjacent'],[2012,2016,2020,2012,2016,2020]]
df_rural0010.columns=[['metro','metro','metro','metro adjacent','metro adjacent','metro adjacent'],[2000,2004,2008,2000,2004,2008]]

df_rural = pd.concat([df_rural0010,df_rural1020],axis=1).dropna()
df_rural=df_rural.sort_index(axis=1)
df_rural.index=df_rural.index.map(int)

#add to total data
total_time_data=pd.concat([total_time_data,df_rural],axis=1)
total_time_data=total_time_data.dropna()

In [ ]:
# values of S&P, Nasdaq and the inflation rate
df_SP500=pd.DataFrame(index=total_time_data.index)
electionyears=[2000,2004,2008,2012,2016,2020]
SPvalues=[1421.22,1130.54,971.31,1417.26,2129.92,3336.25]
for i in range(6):
    df_SP500[electionyears[i]]=np.repeat(SPvalues[i],total_time_data.shape[0])
df_SP500.columns=[np.repeat('SP500',6),electionyears]

df_nasdaq=pd.DataFrame(index=total_time_data.index)
nasdaq=[3399.28,1981.47,1761.09,3003.71,5154.99,11038.66]
for i in range(6):
    df_nasdaq[electionyears[i]]=np.repeat(nasdaq[i],total_time_data.shape[0])
df_nasdaq.columns=[np.repeat('nasdaq',6),electionyears]

df_inflation=pd.DataFrame(index=total_time_data.index)
inflation=[3.4,2.7,3.8,2.1,1.3,1.2]
for i in range(6):
    df_inflation[electionyears[i]]=np.repeat(inflation[i],total_time_data.shape[0])
df_inflation.columns=[np.repeat('inflation',6),electionyears]

total_time_data=pd.concat([total_time_data,df_inflation,df_SP500,df_nasdaq],axis=1)
total_time_data=total_time_data.dropna()

In [ ]:
# To get the presidential events a function is written to webscrape
def url_get_contents(url):
 
    # Opens a website and read its
    # binary contents (HTTP Response Body)
 
    #making request to the website
    req = urllib.request.Request(url=url)
    f = urllib.request.urlopen(req)
    #f=open(req, 'r', encoding = get_encoding_type, errors='ignore')
    #reading contents of the website
    return f.read()
# funcrtion to count number of visits
def retrieve_visits(month,year,person,camp):
    if year==2020:
        url='https://www.democracyinaction.us/'+str(year)+'/' +camp+'/'+str(person)+'cal'+str(month)+str(year)[-2:]+'.html'
    else:
        url='http://p'+str(year)+'.org/'+str(camp)+'/'+str(person)+'cal'+str(month)+str(year)[-2:]+'a.html'
    if month=='08':
        try:
            xhtml = url_get_contents(url).decode('cp1252')
        except Exception:
            return []
    else:
        try:
            xhtml = url_get_contents(url).decode('utf-8')
        except Exception:
            return []
    p = HTMLTableParser()
    p.feed(xhtml)
    temp1=[]
    for i in p.tables:
        for j in i:
            for k in j:
                if len(k)>5:
                    temp1.append(k)
    temp2=[]
    for i in temp1:
        temp2.append(re.split('\s\s', i))
       
    temp3=[]
    for i in temp2:
        for j in i:
            if len(j)>5 and len(j)<50:
                temp3.append(j)
    
    temp4=[]
    for i in temp3:
        temp4.append(re.split('[()]', i))
    
    visits=[]
    
    for i in temp4:
        #print(i)
        for j in i:
            if len(j)>5 and len(j)<50:
                visits.append(j)
    return visits

#now the function above will be used to loop over all elections and candidates
city=pd.read_csv('msc_data_raw/zip_code_database.csv')
df_visits=pd.DataFrame(index=df_education['FIPS Code'],columns=['county','State'])
df_visits['county']=df_education.set_index('FIPS Code')['Area name']
df_visits['State']=df_education.set_index('FIPS Code')['State']
allcandidates=[]
for j in [2000,2004,2008,2012,2016,2020]:
    n=-1
    if j==2000:
        candidates=['bush','cheney','lbush','gore','lieberman','tipper']
    elif j==2004:   
        candidates=['bush','cheney','lbush','kerry','edwards','thk']
    elif j==2008:
        candidates=['mccain','palin','cindy','obama','biden','michelle']
    elif j==2012:
        candidates=['romney','ryan','ann','obama','biden','michelle']
    elif j==2016:
        candidates=['trump','pence','ivana','clinton','kaine','billclinton']
    elif j==2020:
        candidates=['trump','pence','ivana','biden','harris','jillbiden']
    allcandidates.extend(candidates)
    for k in candidates:
        df_visits[k+str(j)]=np.repeat(0,len(df_visits))
        n+=1
        visits=[]
        for i in ['03','04','05','06','07','08','09','10']:
            if n>=3:
                l=candidates[3]
            else:
                l=candidates[0]
            temp=retrieve_visits(i,j,k,l)
            visits.extend(temp)
            
            
            df_visitstemp=pd.DataFrame(index=range(len(visits)),columns=['county','state'])
            for i in range(len(visits)):
                try:
                    df_visitstemp.iloc[i]=re.split(',', visits[i])
                except Exception:
                    df_visitstemp.loc[i]=np.nan
                if df_visitstemp.loc[i,'county']==df_visitstemp.loc[i,'state']:
                    df_visitstemp.loc[i]=np.nan
            df_visitstemp=df_visitstemp.dropna()
            for i in df_visitstemp.index:
                df_visitstemp.loc[i,'state']=df_visitstemp.loc[i,'state'][-2:]
                
                
            for i in range(len(df_visitstemp)):
                county=df_visitstemp.iloc[i,0]
                state=df_visitstemp.iloc[i,1]
                istar=county+' County' 
                if np.sum(np.array(df_visits['county']==county)*np.array(df_visits['State']==state))!=0 or np.sum(np.array(df_visits['county']==istar)*np.array(df_visits['State']==state))!=0:
                    df_visits[k+str(j)]=np.array(df_visits['county']==county)*np.array(df_visits['State']==state)+np.array(df_visits[k+str(j)])
                    df_visits[k+str(j)]=np.array(df_visits['county']==istar)*np.array(df_visits['State']==state)+np.array(df_visits[k+str(j)])
                elif state in set(city.loc[city.loc[:,'primary_city']==county]['state']):
                    real_county=city.loc[city.loc[:,'primary_city']==county][np.array(city.loc[city.loc[:,'primary_city']==county]['state']==state)]['county'].iloc[0]
                    df_visits[k+str(j)]=np.array(df_visits['county']==real_county)*np.array(df_visits['State']==state)+np.array(df_visits[k+str(j)])
            
#putting it in one dataframe
df_visits=df_visits.drop(['county','State'],axis=1)
df_visits.columns=[np.repeat([2000,2004,2008,2012,2016,2020],6),['visits_rep_pres','visits_rep_vp','visits_rep_FL','visits_dem_pres','visits_dem_vp','visits_dem_FL',
                                                                'visits_rep_pres','visits_rep_vp','visits_rep_FL','visits_dem_pres','visits_dem_vp','visits_dem_FL',
                                                                'visits_rep_pres','visits_rep_vp','visits_rep_FL','visits_dem_pres','visits_dem_vp','visits_dem_FL',
                                                                'visits_rep_pres','visits_rep_vp','visits_rep_FL','visits_dem_pres','visits_dem_vp','visits_dem_FL',
                                                                'visits_rep_pres','visits_rep_vp','visits_rep_FL','visits_dem_pres','visits_dem_vp','visits_dem_FL',
                                                                'visits_rep_pres','visits_rep_vp','visits_rep_FL','visits_dem_pres','visits_dem_vp','visits_dem_FL']]

df_total_visits=pd.DataFrame(index=df_visits.index, columns=[np.repeat([2000,2004,2008,2012,2016,2020],2),
                                                            ['visits_dem','visits_rep','visits_dem','visits_rep','visits_dem','visits_rep',
                                                            'visits_dem','visits_rep','visits_dem','visits_rep','visits_dem','visits_rep']])
for i in [2000,2004,2008,2012,2016,2020]:
    df_total_visits.loc[:,(i,'visits_rep')]=np.sum(df_visits[i].iloc[:,:3],axis=1)
    df_total_visits.loc[:,(i,'visits_dem')]=np.sum(df_visits[i].iloc[:,3:],axis=1)
df_total_visits=df_total_visits.swaplevel(axis=1).sort_index(axis=1)
df_visits=df_visits.swaplevel(axis=1).sort_index(axis=1)
# the corrected visits is the visits divided by eligible voters
df_corrected_visits_dem=pd.DataFrame(index=df_total_visits.index,columns=[np.repeat('corrected total dem visits',6),
                                                                         [2000,2004,2008,2012,2016,2020]])
temp = pd.concat([df_total_visits.iloc[:,:6],df_eligibevoters],axis=1).dropna()
temp
df_corrected_visits_dem=temp['visits_dem']/temp['eligible voters']*50000
df_corrected_visits_dem.columns=[np.repeat('corrected total dem visits',6),
                                [2000,2004,2008,2012,2016,2020]]
df_corrected_visits_rep=pd.DataFrame(index=df_total_visits.index,columns=[np.repeat('corrected total rep visits',6),
                                                                         [2000,2004,2008,2012,2016,2020]])
temp = pd.concat([df_total_visits.iloc[:,6:],df_eligibevoters],axis=1).dropna()
temp
df_corrected_visits_rep=temp['visits_rep']/temp['eligible voters']*5000
df_corrected_visits_rep.columns=[np.repeat('corrected total rep visits',6),
                                [2000,2004,2008,2012,2016,2020]]

total_time_data=pd.concat([total_time_data,df_total_visits,df_visits,df_corrected_visits_rep,df_corrected_visits_dem],axis=1)
total_time_data=total_time_data.dropna()

In [ ]:
#save the cleaned data
#total_time_data.to_csv('msc_thesis/data/totaldata')